In [ ]:
%env OPENAI_API_KEY = ""
%env OPENAI_API_KEY = ""

env: OPENAI_API_KEY=


In [ ]:
!pip install langchain==0.0.156
!pip install openai
!pip install --upgrade diffusers accelerate transformers
!pip install duckduckgo-search
# ==== PDF ==== #
!pip install pypdf
!pip install chromadb
!pip install pycryptodome==3.15.0
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.6/727.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56

In [ ]:
# fix colab unknown problem, 從網路上抄下來的未知code.
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

persist_dir = './chroma_storage'
#* Loading
loader = PyPDFLoader('./Assignment3.pdf')
pages = loader.load_and_split() # loader & text_spliter
#* Embedding
embeddings = OpenAIEmbeddings() # There has a better way.

vectorstore = Chroma.from_documents(documents=pages,
                                    embedding=embeddings,
                                    persist_directory=persist_dir)
vectorstore.persist()



# tool testing
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

@tool("Search for infomation of My AI homework3")
def pdf_tool(query: str) -> str:
    """Useful only for when the question is specific only about AI homework3"""

    persist_dir = './chroma_storage'



    #* Loading
    loader = PyPDFLoader('./Assignment3.pdf')
    pages = loader.load_and_split() # loader & text_spliter
    #* Embedding
    embeddings = OpenAIEmbeddings() # There has a better way.

    vectorDB = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    #* Searching
    semantic_search = vectorDB.similarity_search(query, k=4)

    return semantic_search

# ===== init agent ===== #
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory # Memory
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""




llm = HuggingFaceHub(repo_id="databricks/dolly-v2-3b", model_kwargs={"temperature":0, "max_length":64})
# llm = OpenAI(temperature=0)

tools=[ct.chat_tool,
       ct.search_tool,
       ct.text_to_image_api,
       ct.text_to_anime_liked_image_api,
       ct.text_to_ghibli_style_drawing_api]

memory = ConversationBufferMemory(memory_key="chat_history") # return_messages=False, agent 才會看

zero_shot_agent = initialize_agent(
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iteration=1,
    early_stopping_method="generate"
)

while True:
  prompt = input()
  reply = zero_shot_agent.run(prompt)
  print(reply)